In [1]:
import datasets
import torch
from transformers import BeitForImageClassification, BeitFeatureExtractor, BeitImageProcessor
import numpy as np

base_model_name = "microsoft/beit-base-patch16-224-pt22k"

processor = BeitImageProcessor.from_pretrained(base_model_name)

def crop_image(img):
    width, height = img.size
    target_size = min(width, height)
    left = max(0, (width - target_size ) // 2)
    right = left + target_size
    top = max(0, (height - target_size ) // 2)
    bottom = top + target_size
    return img.crop((left, top, right, bottom)).convert('RGB')

def process_example(example):
    img = crop_image(example['image'])
    inputs = processor(img, return_tensors='pt')
    inputs['labels'] = example['label']
    return inputs

def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = processor([crop_image(x) for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

ds = datasets.load_from_disk("../../data/diffusion_and_real/")

prepared_ds = ds.with_transform(transform)


def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = datasets.load_metric("f1")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

labels = ["negative","positive"]

model = BeitForImageClassification.from_pretrained(
    base_model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
).to("cuda")

C:\Users\Tim\AppData\Local\Temp/ipykernel_16212/3893446471.py:44: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("f1")
Some weights of the model checkpoint at microsoft/beit-base-patch16-224-pt22k were not used when initializing BeitForImageClassification: ['layernorm.weight', 'layernorm.bias', 'lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing BeitForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BeitForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

In [2]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./BEiT-diff-detect",
  per_device_train_batch_size=32, # consider 16 or higher  # add dropout
  evaluation_strategy="steps",
  num_train_epochs=3,
  fp16=True,
  save_steps=6000,
  eval_steps=48,
  logging_steps=1125,
  learning_rate=2e-4,
  save_total_limit=3,
  logging_first_step=True,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["vali"],
    tokenizer=processor,
)

In [3]:
# start the training
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

c:\Users\Tim\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 1/1689 [00:10<4:59:14, 10.64s/it]

{'loss': 0.7526, 'learning_rate': 0.0002, 'epoch': 0.0}


                                                   
  3%|▎         | 48/1689 [04:54<1:40:33,  3.68s/it]

{'eval_loss': 0.501287043094635, 'eval_f1': 0.7867768595041322, 'eval_runtime': 111.669, 'eval_samples_per_second': 8.955, 'eval_steps_per_second': 1.119, 'epoch': 0.09}


                                                    
  6%|▌         | 96/1689 [06:43<23:04,  1.15it/s]

...

{'eval_loss': 0.1516994833946228, 'eval_f1': 0.9538461538461538, 'eval_runtime': 11.9429, 'eval_samples_per_second': 83.732, 'eval_steps_per_second': 10.466, 'epoch': 2.9}


                                                   
 99%|█████████▉| 1680/1689 [36:40<00:06,  1.29it/s]

{'eval_loss': 0.15105971693992615, 'eval_f1': 0.9538461538461538, 'eval_runtime': 13.9387, 'eval_samples_per_second': 71.743, 'eval_steps_per_second': 8.968, 'epoch': 2.98}


100%|██████████| 1689/1689 [36:47<00:00,  1.31s/it]


{'train_runtime': 2207.1281, 'train_samples_per_second': 24.466, 'train_steps_per_second': 0.765, 'train_loss': 0.20243481533847807, 'epoch': 3.0}
***** train metrics *****
  epoch                    =        3.0
  train_loss               =     0.2024
  train_runtime            = 0:36:47.12
  train_samples_per_second =     24.466
  train_steps_per_second   =      0.765


In [4]:
# eval
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

100%|██████████| 125/125 [00:12<00:00, 10.19it/s]

***** eval metrics *****
  epoch                   =        3.0
  eval_f1                 =     0.9617
  eval_loss               =     0.1135
  eval_runtime            = 0:00:12.38
  eval_samples_per_second =     80.761
  eval_steps_per_second   =     10.095
